In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
from erddapy import ERDDAP
import json
import re

Configure the ERDDAP to search

In [2]:
ra = 'CARICOOS'

with open('utils/ra_erddaps.json') as f:
    urls = json.load(f)

server = urls[ra.lower()]

print(server)

e = ERDDAP(server=server, protocol="tabledap")

http://dm3.caricoos.org:8002/erddap


Pull in the processed inventory

In [3]:
#
# df_cruise = df[df['Station Description'] == 'Chesapeake Bay WQ Cruise Data ']
#
#df_aws = df[df['Station ID'] == 'Humboldt']
# #url = 'http://tds.glos.us/thredds/dodsC/buoy_agg_standard/OMOECC_E1/OMOECC_E1.ncml'
# #url = 'http://tds.glos.us/thredds/dodsC/buoy_agg_standard/45186/45186.ncml'
# #url = 'http://tds.glos.us/thredds/dodsC/buoy_agg_standard/bgsusd2/bgsusd2.ncml'
# url = 'http://oos.soest.hawaii.edu/thredds/dodsC/hioos/nss/ns12agg'
# print(url)
# ds = xr.open_dataset(url)
# #ds = netCDF4.Dataset(url,'r')
# title = ds.title
# start_time = np.datetime_as_string(ds.time.min().values, unit='D')
# end_time = np.datetime_as_string(ds.time.max().values, unit='D')
# print('Duration: %s - %s' % (np.datetime_as_string(ds.time.min().values, unit='D'),
#                              np.datetime_as_string(ds.time.max().values, unit='D'))
#       )

# Try glos web server
# url = 'https://glbuoys.glos.us/tools/export?ftype=csv&data_type=buoy&units=eng&locs=OMOECC_E1&params=Water_Temperature_at_Surface|dissolved_oxygen_saturation|water_conductivity|ysi_turbidity&tperiod=custom&date_start=2020-01-01&date_end=2020-12-31&avg_ivld=none'
# df = erddapy.ERDDAP.to_pandas(url)

## using ERDDAP to look for stations
# check out http://data.glos.us/erddap/tabledap/allDatasets.htmlTable?datasetID%2Ctitle%2CminTime%2CmaxTime&maxTime%3E=2020-01-01&maxTime%3C=2020-12-31&orderBy(%22maxTime%22)
# that lists out all the GLOS stations with the maximum time of observations within the year 2020 (on their ERDDAP).


ra_clean = re.sub(r'[0-9]', '', ra) # remove number from RA 

df = pd.read_excel('2023/data/processed/%s.xlsx' % ra_clean.upper(), dtype=str)
df

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude (dec deg),Longitude (dec deg),Platform Type,"Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)","Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",Platform Funder/Sponsor,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,"Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].",Additional Notes,CARICOOS Contact\n(Name and Email)
0,CARICOOS,PR1,42085,Ponce Buoy,NaN,17.8687,-66.532,Moored buoy,06/2009 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 16) m, sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu)
1,CARICOOS,PR2,41053,San Juan Buoy,NaN,17.4743,-66.0994,Moored buoy,07/2010 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 29) m, sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu)
2,CARICOOS,PR3,41056,Vieques Buoy,NaN,18.2609,-65.464,Moored buoy,09/2013 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 28 m), sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu)
3,CARICOOS,VI1,41052,St. John Buoy,NaN,18.2491,-64.7637,Moored buoy,04/2011 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 40 m), sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu)
4,CARICOOS,VI2,41051,North of St. Thomas,NaN,18.4761,-65.1568,Moored buoy,08/2017-09/2017,Offline,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Damaged by Hurricane Irma. A new oceanographic...,Julio Morell \n(julio.morell@upr.edu)
5,CARICOOS,VIA,41051,North of St. Thomas,NaN,18.2574,-65.004,Moored buoy,03/2013-03/2015,No,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,University of Virgin Islands,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Redeployed in another location. Now is identif...,Julio Morell \n(julio.morell@upr.edu)
6,CARICOOS,181p1,41115,Rincón Waverider Buoy,NaN,18.3764,-67.2799,Moored buoy,05/2011 - present,Operational,CARICOOS/CDIP,N,"Coastal Data Information Program, SIO/UCSD/CAR...",Academia,CARICOOS,Coastal Data Information Program,"sea_surface_wave_significant_height, sea_surfa...",NaN,Miguel Canals\n(miguelf.canals@upr.edu)
7,NREL/CARICOOS,249p1,41121,Arecibo Waverider Buoy,NaN,18.4909,-66.7005,Moored buoy,05/2021 - present,Operational,CARICOOS/CDIP/NREL,Yf,"Coastal Data Information Program, SIO/UCSD/CAR...",Government-Federal/Academia,CARICOOS,Coastal Data Information Program,"sea_surface_wave_significant_height, sea_surfa...",NaN,Miguel Canals\nmiguelf.canals@upr.edu
8,"CARICOOS, OAP",MAPCO2,NCEI Accession 0117354,La Parguera MapCO2 buoy,NCEI Accession 0117354,17.953666666666667,-66.0512,Moored buoy,2009 - present,Operational,PMEL/CARICOOS,Yp,PMEL,Academia,CARICOOS,PMEL/CARICOOS,"SST, SSS, Atm. press, xCO2 water, xCO2 air, fC...",NaN,Julio Morell \n(julio.morell@upr.edu)
9,CARICOOS,PUNTAS,PTRP4,"Rincón, PR",NaN,18.3667,-67.2514,Fixed,11/2012 - prese

Search the ERDDAP server for a single station of interest - more of a test

In [4]:
# df['Station ID'].loc[df['Station ID'].isna()] = df.loc[df['Station ID'].isna(),'Dataset ID']

# df['Station ID']

In [5]:
# df_wf = df[df['Station ID'] == 'Jug_Bay']

# # for SCCOOS need to search for 'Station Long Name' as 'Station ID' is NA for most entries
# search_for = df_wf['Station ID'].to_string()

# search_url = e.get_search_url(search_for=search_for, response="csv")

# try:
#     resp = pd.read_csv(search_url)
#     print(resp['Station ID'])
# except:
#     pass

Rip through all the datasets in the inventory and see if they exist on the RA erddap. Process checks the following order:

1. `Station ID`, then 
2. `Station Long Name`, then
3. `WMO ID or NWS/CMAN ID`

We first check if the value is `nan`. If so, we punt as that will return all datasets from the ERDDAP service.

In [6]:
not_found = []
found = dict()

exclude = ' -gliders'

#search_order = ['Station ID', 'Station Long Name', 'WMO ID or NWS/CMAN ID']

# GLOS
search_order = ['Station ID', 'Station Long Name', 'WMO ID or NWS/CMAN ID']
# SCCOOS
#search_order = ['Station Long Name', 'WMO ID or NWS/CMAN ID', 'Station Description']

for search1 in df[search_order[0]].astype(str):
    
    if search1 == "nan":
        print("punting with %s = nan\n" % search_order[0])
        not_found.append(search1)
        continue
    
    try:
        print('searching \"%s\" = \"%s\"' % (search_order[0], search1))
        search_url = e.get_search_url(search_for=search1+exclude, response="csv")
        resp = pd.read_csv(search_url)
        
        print("Found \"%s\": %s" % (search1,resp['Dataset ID'].values.tolist()))
        
        found[search1] = {'Dataset ID': resp['Dataset ID'].values.tolist()}
        
    except:
        
        print('no dataset matching \"%s\"' % search1)
        search2 = df.loc[df[search_order[0]] == search1, search_order[1]].values.tolist()[0]
        
        if search2 == "nan":
                print("punting with %s = nan\n" % search_order[1])
                not_found.append(search1)
                continue
                
        try:
            
            print('searching \"%s\" = \"%s\"' % (search_order[1], search2))
            search_url = e.get_search_url(search_for=search2+exclude, response="csv")
            resp = pd.read_csv(search_url)
            print("Found \"%s\": %s" % (search2,resp['Dataset ID'].values.tolist()))
            
            found[search1] = {'Dataset ID': resp['Dataset ID'].values.tolist()}
            
        except:

            search3 = str(df.loc[df[search_order[0]] == search1, search_order[2]].values.tolist()[0])
            
            if search3 == "nan":
                print("punting with \"%s\" = nan\n" % search_order[2])
                not_found.append(search1)
                continue
            
            try:
                print('searching \"%s\" = \"%s\"' % (search_order[2], search3))
                search_url = e.get_search_url(search_for=search3+exclude, response="csv")
                resp = pd.read_csv(search_url)
                print("Found \"%s\": %s" % (search3,resp['Dataset ID'].values.tolist()))
                
                found[search1] = {'Dataset ID': resp['Dataset ID'].values.tolist()}
                
            except:
                print("Couldn't find:", [search1, search2, search3])
                not_found.append(search1)
        
    print("\n")

searching "Station ID" = "PR1"
Found "PR1": ['PR1_doppler_5e96_75ee_2a90', 'PR1_sbe_f2d2_f08a_3de6', 'PR1_triaxys_ad35_042f_ee54', 'PR1_met_02ff_fa06_8f89']


searching "Station ID" = "PR2"
Found "PR2": ['PR2_doppler_34d7_c61a_a162', 'PR2_sbe_4bd8_cdd4_f3a1', 'PR2_triaxys_8b12_c1a2_3611', 'buoy_erddap_12bc_0293_5a40']


searching "Station ID" = "PR3"
Found "PR3": ['PR3_doppler_b3ca_6da8_3d0e', 'PR3_sbe_dea3_226c_8bd7', 'PR3_met_cef1_bff2_f25b', 'PR3_waves_3d94_aeda_8139']


searching "Station ID" = "VI1"
Found "VI1": ['VI1_doppler_200f_7ac1_2075', 'VI1_sbe_f867_8696_97f1', 'VI1_triaxys_ee48_77fd_a313', 'VI1_met_60d8_4131_a3f0']


searching "Station ID" = "VI2"
Found "VI2": ['VI2_doppler_36f0_5d1f_762a', 'VI2_waves_1f59_412f_3b40', 'VI2_met_599b_402a_1961', 'VI2_wqm_1e83_3b14_057e']


searching "Station ID" = "VIA"
Found "VIA": ['VIA_doppler_7454_e002_60ee', 'VIA_wqm_5e00_012a_371c', 'VIA_waves_3dae_8bcb_1992', 'VIA_met_596a_fd3e_713e', 'Mean_847a_dde4_02e4', 'Radial_d47f_bb7f_4559', 'M

Print the stations that we couldn't find.

In [7]:
len(not_found)

0

In [8]:
len(found)

33

Show me the record that has NaN for the first search element.

In [9]:
df.loc[df[search_order[0]].isna()]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude (dec deg),Longitude (dec deg),Platform Type,"Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)","Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",Platform Funder/Sponsor,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,"Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].",Additional Notes,CARICOOS Contact\n(Name and Email)


Now show me the stations, with identifiers, that we couldn't find.

In [10]:
df[df[search_order[0]].isin(not_found)]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude (dec deg),Longitude (dec deg),Platform Type,"Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)","Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",Platform Funder/Sponsor,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,"Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].",Additional Notes,CARICOOS Contact\n(Name and Email)


Print out all the dataset IDs for the datasets that matched our criteria.

In [11]:
found

{'PR1': {'Dataset ID': ['PR1_doppler_5e96_75ee_2a90',
   'PR1_sbe_f2d2_f08a_3de6',
   'PR1_triaxys_ad35_042f_ee54',
   'PR1_met_02ff_fa06_8f89']},
 'PR2': {'Dataset ID': ['PR2_doppler_34d7_c61a_a162',
   'PR2_sbe_4bd8_cdd4_f3a1',
   'PR2_triaxys_8b12_c1a2_3611',
   'buoy_erddap_12bc_0293_5a40']},
 'PR3': {'Dataset ID': ['PR3_doppler_b3ca_6da8_3d0e',
   'PR3_sbe_dea3_226c_8bd7',
   'PR3_met_cef1_bff2_f25b',
   'PR3_waves_3d94_aeda_8139']},
 'VI1': {'Dataset ID': ['VI1_doppler_200f_7ac1_2075',
   'VI1_sbe_f867_8696_97f1',
   'VI1_triaxys_ee48_77fd_a313',
   'VI1_met_60d8_4131_a3f0']},
 'VI2': {'Dataset ID': ['VI2_doppler_36f0_5d1f_762a',
   'VI2_waves_1f59_412f_3b40',
   'VI2_met_599b_402a_1961',
   'VI2_wqm_1e83_3b14_057e']},
 'VIA': {'Dataset ID': ['VIA_doppler_7454_e002_60ee',
   'VIA_wqm_5e00_012a_371c',
   'VIA_waves_3dae_8bcb_1992',
   'VIA_met_596a_fd3e_713e',
   'Mean_847a_dde4_02e4',
   'Radial_d47f_bb7f_4559',
   'Mean_04e5_40e7_ed1d',
   'Radial_76a9_c913_9300',
   'Parguera_C

Check if each found dataset has coverage for specified CY

In [12]:
valid_datasets = []

cy = '2023'

for key in found:
    
    dataset_id = found[key]['Dataset ID']
    
    if len(dataset_id)>1:
        
        for dataset in dataset_id:
            
            e.dataset_id = dataset
    
            e.constraints = {
                 "time>=": "{}-01-01".format(cy)
            }
        
            e.variables = ['time']
    
            try:
                df_data = e.to_pandas(parse_dates=True)
                valid_datasets.extend([key])
                
            except:
                pass

                #print("No data for 2021 for %s: dataset ID %s" % (key,e.dataset_id))

    else:
        
        e.dataset_id = dataset_id[0]
    
        e.constraints = {
                         "time>=": "{}-01-01".format(cy)
                        }
        
        e.variables = ['time']
        
        try:
            
            df_data = e.to_pandas(parse_dates=True)

            valid_datasets.extend([key])

        except:
            pass

            #print("No data for 2021 for %s: dataset ID %s" % (key,e.dataset_id))

            
#for valid in sorted(set(valid_datasets)):
    
#     # drop qc vars
#     cols = [c for c in df_data.columns if 'qc' not in c]
#     cols = [c for c in cols if 'QARTOD' not in c]

#     df_data = df_data[cols]

#     # set index for plotting
#     df_data = df_data.set_index(df_data['time (UTC)'])

#     # plot
#     #df_data.plot(subplots=True, title=e.dataset_id,figsize=(20,20))

#     start_time = df_data['time (UTC)'].min()
#     end_time = df_data['time (UTC)'].max()

#     print('Dataset %s' % e.dataset_id)
#     print('Duration: %s - %s' % (start_time, end_time))

no_cov_datasets = found.keys() - sorted(set(valid_datasets))

print('Datasets without coverage for {}:\n{}'.format(cy,no_cov_datasets))

Datasets without coverage for 2023:
{'VI2', 'XWGO', 'XAMA', 'XSAV', 'MAPCO2', 'PUNTAS'}


Show me the inventory metadata about the invalid stations:

In [13]:
bad_stations = df.loc[(df[search_order[0]].isin(no_cov_datasets)) | df[search_order[0]].isin(not_found)]

df['erddap_not_avail'] = df[search_order[0]].isin(bad_stations[search_order[0]])

df.loc[df['erddap_not_avail']]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude (dec deg),Longitude (dec deg),Platform Type,"Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)","Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",Platform Funder/Sponsor,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,"Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].",Additional Notes,CARICOOS Contact\n(Name and Email),erddap_not_avail
4,CARICOOS,VI2,41051,North of St. Thomas,NaN,18.4761,-65.1568,Moored buoy,08/2017-09/2017,Offline,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Damaged by Hurricane Irma. A new oceanographic...,Julio Morell \n(julio.morell@upr.edu),True
8,"CARICOOS, OAP",MAPCO2,NCEI Accession 0117354,La Parguera MapCO2 buoy,NCEI Accession 0117354,17.953666666666667,-66.0512,Moored buoy,2009 - present,Operational,PMEL/CARICOOS,Yp,PMEL,Academia,CARICOOS,PMEL/CARICOOS,"SST, SSS, Atm. press, xCO2 water, xCO2 air, fC...",NaN,Julio Morell \n(julio.morell@upr.edu),True
9,CARICOOS,PUNTAS,PTRP4,"Rincón, PR",NaN,18.3667,-67.2514,Fixed,11/2012 - present,Offline,CARICOOS,Yf,CARICOOS,Academia/Non-Profit,CARICOOS,NDBC / CARICOOS,"wind_speed (15 m), wind_speed_of_gust (15 m), ...",Issues with the data logger. CARICOOS will rep...,Patricia Chardón-Maldonado (patricia.chardon@u...,True
16,CARICOOS,XAMA,TIMA,"Charlotte Amalie Harbor, USVI",NaN,18.3303,-64.9264,Fixed,2009 - 2015,No,Weatherflow /CARICOOS,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",NaN,Patricia Chardón-Maldonado (patricia.chardon@u...,True
27,CARICOOS,XSAV,TIAC,"Savanah Island, USVI",NaN,18.33737,-65.083,Fixed,2009 - 2017,No,Weatherflow /CARICOOS,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Decomissioned by Weatherflow because it got da...,Patricia Chardón-Maldonado (patricia.chardon@u...,True
28,CARICOOS,XWGO,TIGO,"Crown Mountain, USVI",NaN,18.35537,-64.96693,Fixed,2009 - 2017,No,Weatherflow /CARICOOS,Yf,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Damaged by Hurricane Maria. A new station will...,Patricia Chardón-Maldonado (patricia.chardon@u...,True


Invert the `erddap_not_avail` column and drop it.

In [14]:
df['erddap_avail'] = ~df['erddap_not_avail']

df.drop(columns='erddap_not_avail', inplace=True)

df

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude (dec deg),Longitude (dec deg),Platform Type,"Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)","Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",Platform Funder/Sponsor,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,"Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].",Additional Notes,CARICOOS Contact\n(Name and Email),erddap_avail
0,CARICOOS,PR1,42085,Ponce Buoy,NaN,17.8687,-66.532,Moored buoy,06/2009 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 16) m, sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu),True
1,CARICOOS,PR2,41053,San Juan Buoy,NaN,17.4743,-66.0994,Moored buoy,07/2010 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 29) m, sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu),True
2,CARICOOS,PR3,41056,Vieques Buoy,NaN,18.2609,-65.464,Moored buoy,09/2013 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 28 m), sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu),True
3,CARICOOS,VI1,41052,St. John Buoy,NaN,18.2491,-64.7637,Moored buoy,04/2011 - present,Operational,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 40 m), sea_water_to_dir...",NaN,Julio Morell \n(julio.morell@upr.edu),True
4,CARICOOS,VI2,41051,North of St. Thomas,NaN,18.4761,-65.1568,Moored buoy,08/2017-09/2017,Offline,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Damaged by Hurricane Irma. A new oceanographic...,Julio Morell \n(julio.morell@upr.edu),False
5,CARICOOS,VIA,41051,North of St. Thomas,NaN,18.2574,-65.004,Moored buoy,03/2013-03/2015,No,CARICOOS,Yf,Neil Pettigrew/CARICOOS,Academia,University of Virgin Islands,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Redeployed in another location. Now is identif...,Julio Morell \n(julio.morell@upr.edu),True
6,CARICOOS,181p1,41115,Rincón Waverider Buoy,NaN,18.3764,-67.2799,Moored buoy,05/2011 - present,Operational,CARICOOS/CDIP,N,"Coastal Data Information Program, SIO/UCSD/CAR...",Academia,CARICOOS,Coastal Data Information Program,"sea_surface_wave_significant_height, sea_surfa...",NaN,Miguel Canals\n(miguelf.canals@upr.edu),True
7,NREL/CARICOOS,249p1,41121,Arecibo Waverider Buoy,NaN,18.4909,-66.7005,Moored buoy,05/2021 - present,Operational,CARICOOS/CDIP/NREL,Yf,"Coastal Data Information Program, SIO/UCSD/CAR...",Government-Federal/Academia,CARICOOS,Coastal Data Information Program,"sea_surface_wave_significant_height, sea_surfa...",NaN,Miguel Canals\nmiguelf.canals@upr.edu,True
8,"CARICOOS, OAP",MAPCO2,NCEI Accession 0117354,La Parguera MapCO2 buoy,NCEI Accession 0117354,17.953666666666667,-66.0512,Moored buoy,2009 - present,Operational,PMEL/CARICOOS,Yp,PMEL,Academia,CARICOOS,PMEL/CARICOOS,"SST, SSS, Atm. press, xCO2 water, xCO2 air, fC...",NaN,Julio Morell \n(julio.morell@upr.edu),False
9,CARICOOS,PUNTAS,P

Make a plot of all points and color based on ERDDAP available or not.

In [15]:
import geopandas
import matplotlib.pyplot as plt

#df = df.loc[df['']]

df.rename(columns=
{'Variable Names + water column depth of measurement in meters [CF_name (# m, # m) or CF_name (mult) or CF_name (# depths)].':
     'Variable Names',
 'Station Deployment (mm/yyyy, yyyy, < 5 yr, > 5 yr)': 'Station Deployment',
 'Longitude (dec deg)': 'Longitude',
 'Latitude (dec deg)': 'Latitude'},
              inplace=True)

df['Station Deployment'] = \
    df['Station Deployment'].astype(str)


gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df['Longitude'], df['Latitude']))

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

ax = world.plot(
    color='whitesmoke', 
    edgecolor='grey', 
    figsize=(25,20),
    )

gdf.plot( ax=ax,
         #column='erddap_avail', 
         categorical=True, 
         cmap = 'Set1',
         markersize=10, 
         legend=True,
         legend_kwds={'title': "ERDDAP Available?"},
         k=2,
        )



minx, miny, maxx, maxy = gdf.total_bounds
ax.set_xlim(minx-1, maxx+1)
ax.set_ylim(miny-1, maxy+1)

plt.title('{} station distribution'.format(ra))

plt.show()

Make an interactive map.

When you click on a point the return is the information from the inventory.

In [16]:
gdf_final = gdf.set_crs(epsg=4326)

gdf_final.loc[~gdf_final['geometry'].is_empty == True].drop(columns='Variable Names').explore('erddap_avail',cmap = 'Set1')


## Show me some bad coordinates

_if there are any_

In [17]:
gdf_final.loc[gdf_final['geometry'].is_empty == True]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude,Longitude,Platform Type,Station Deployment,"Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",...,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,Variable Names,Additional Notes,CARICOOS Contact\n(Name and Email),erddap_avail,geometry


In [18]:
# import urllib.request
# for url in gdf_final.loc[gdf_final['erddap_not_avail'],'Station Description'].str.replace('url: ',''):
#     print(url)
#     try:
#         urllib.request.urlopen(url).getcode()
#     except:
#         print("URL broken: %s" % url)
#         print(gdf_final.loc[gdf_final['Station Description'].str.contains(url,regex=False),'Station ID'])

# Show me the stations where ERDDAP doesn't exist or provide a valid response.

Reasons:
* Station not found on RA erddap
* dataset found but no data for this CY.

In [19]:
gdf_final.loc[~gdf_final['erddap_avail']]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude,Longitude,Platform Type,Station Deployment,"Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",...,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,Variable Names,Additional Notes,CARICOOS Contact\n(Name and Email),erddap_avail,geometry
4,CARICOOS,VI2,41051,North of St. Thomas,NaN,18.4761,-65.1568,Moored buoy,08/2017-09/2017,Offline,...,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Damaged by Hurricane Irma. A new oceanographic...,Julio Morell \n(julio.morell@upr.edu),False,POINT (-65.15680 18.47610)
8,"CARICOOS, OAP",MAPCO2,NCEI Accession 0117354,La Parguera MapCO2 buoy,NCEI Accession 0117354,17.953666666666667,-66.0512,Moored buoy,2009 - present,Operational,...,Yp,PMEL,Academia,CARICOOS,PMEL/CARICOOS,"SST, SSS, Atm. press, xCO2 water, xCO2 air, fC...",NaN,Julio Morell \n(julio.morell@upr.edu),False,POINT (-66.05120 17.95367)
9,CARICOOS,PUNTAS,PTRP4,"Rincón, PR",NaN,18.3667,-67.2514,Fixed,11/2012 - present,Offline,...,Yf,CARICOOS,Academia/Non-Profit,CARICOOS,NDBC / CARICOOS,"wind_speed (15 m), wind_speed_of_gust (15 m), ...",Issues with the data logger. CARICOOS will rep...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-67.25140 18.36670)
16,CARICOOS,XAMA,TIMA,"Charlotte Amalie Harbor, USVI",NaN,18.3303,-64.9264,Fixed,2009 - 2015,No,...,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",NaN,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-64.92640 18.33030)
27,CARICOOS,XSAV,TIAC,"Savanah Island, USVI",NaN,18.33737,-65.083,Fixed,2009 - 2017,No,...,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Decomissioned by Weatherflow because it got da...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-65.08300 18.33737)
28,CARICOOS,XWGO,TIGO,"Crown Mountain, USVI",NaN,18.35537,-64.96693,Fixed,2009 - 2017,No,...,Yf,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Damaged by Hurricane Maria. A new station will...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-64.96693 18.35537)


# Do some checking against the IOOS catalog

only for datasets that we couldn't find in ERDDAP (per above)

In [20]:
from ckanapi import RemoteCKAN
import xarray as xr

problem_datasets = []

ua = 'ckanapiioos/1.0 (+https://ioos.us/)'

#ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua)
ioos_catalog

gdf_ckan = gdf_final.loc[~gdf_final['erddap_avail']]


for station in gdf_ckan['Station ID'].dropna():

    RA = gdf_ckan.loc[gdf_ckan['Station ID']==station,'RA'].tolist()[0]
    
    print('Station "{}" + RA "{}"'.format(station.lower(), RA.lower()))
    
    # build RA tag query
    fq = '+organization:{}'.format(RA.lower())

    # build free text query
    q = "\""+station.lower()+"\""+" -glider"

    query = ioos_catalog.address + '/dataset/?' + 'q=' + q + '&organization=' + RA.lower()
    
    print('Query: {}'.format(query.replace(" ","%20")))

    datasets = ioos_catalog.action.package_search(q=q, fq=fq)

    

    if datasets['count'] > 0:
        print('\tfound {} dataset(s):'.format(datasets['count']))
        
        for dataset in datasets['results']:
            print('\t',dataset['title'])
            
            
            if dataset['resources']:
                print('\t',dataset['resources'][0]['url'])

                start_date = next(item for item in dataset["extras"] if item["key"] == "temporal-extent-begin")['value']
                end_date = next(item for item in dataset["extras"] if item["key"] == "temporal-extent-end")['value']

                print('\t','Coverage Meta:\n\t\t', start_date,' - ',end_date,'\n')

                if 'thredds' in dataset['resources'][0]['url']:
                    ds = xr.open_dataset(dataset['resources'][0]['url'])

                    time_max = ds.time.max().values

                    time_min = ds.time.min().values


                    print('\tCoverage Data:\n\t\t', time_min,' - ',time_max, '\n')

                    if pd.to_datetime(end_date) < pd.to_datetime(time_max, utc=True):
                        test_date = pd.to_datetime(time_max, utc=True)

                else:
                    test_date = pd.to_datetime(end_date, utc=True)

                if test_date > pd.to_datetime('{}-01-01'.format(cy), utc=True):
                    print('\t{} is valid.'.format(station))
                    break
                else:
                    print('\t{} is not valid.'.format(station))

                    problem_datasets.append(station)
            
            else:
                print('\t{} missing additional metadata'.format(station))
                problem_datasets.append(station)
    else:
        print("\tNo datasets found!")
        problem_datasets.append(station)

Station "vi2" + RA "caricoos"
Query: https://data.ioos.us/dataset/?q="vi2"%20-glider&organization=caricoos
	No datasets found!
Station "mapco2" + RA "caricoos, oap"
Query: https://data.ioos.us/dataset/?q="mapco2"%20-glider&organization=caricoos,%20oap
	No datasets found!
Station "puntas" + RA "caricoos"
Query: https://data.ioos.us/dataset/?q="puntas"%20-glider&organization=caricoos
	found 1 dataset(s):
	 PTRP4 CARICOOS Meteorological Station
	 http://dm2.caricoos.org/erddap/tabledap/ptrp4_c6ab_61c7_a39a
	 Coverage Meta:
		 2015-12-21T00:00:00Z  -  2022-04-07T23:20:00Z 

	PUNTAS is not valid.
Station "xama" + RA "caricoos"
Query: https://data.ioos.us/dataset/?q="xama"%20-glider&organization=caricoos
	found 2 dataset(s):
	 XAMA CARICOOS-WeatherFlow Meteorological Station
	 http://dm2.caricoos.org/erddap/tabledap/XAMA_669c_7da2_f8f3
	 Coverage Meta:
		 2012-02-24T04:00:00Z  -  2015-04-01T04:00:00Z 

	XAMA is not valid.
	 XAMA  - Charlotte Amalie Harbor, USVI Aggregated Meteorological Obse

Check the datasets that didn't have coverage in ERDDAP or through IOOS catalog and see if they are **Currently Operational?**

In [21]:
gdf_ckan.loc[gdf_ckan['Station ID'].isin(problem_datasets)]

,RA,Station ID,WMO ID or NWS/CMAN ID,Station Long Name,Station Description (Optional),Latitude,Longitude,Platform Type,Station Deployment,"Currently Operational? (Y, N, O, U) \nY = Yes (this includes seasonal buoys intended for redeployment). N = No (it was uninstalled/removed, or is offline with no intent to repair). O = Offline (temporary outage, with intent to repair or redeploy). U = Unknown (e.g. funding uncertainty for repair/redeployment, or operator's plans are unknown). If O or U, please indicate why in the notes column.)",...,RA Funding Involvement \n(Yf = Fully funds. Yp = Partially funds. N = No funding),Platform Operator/Owner,Operator Sector,Platform Maintainer,Data Manager,Variable Names,Additional Notes,CARICOOS Contact\n(Name and Email),erddap_avail,geometry
4,CARICOOS,VI2,41051,North of St. Thomas,NaN,18.4761,-65.1568,Moored buoy,08/2017-09/2017,Offline,...,Yf,Neil Pettigrew/CARICOOS,Academia,CARICOOS/University of Maine,"Department of Physical Oceanography, School of...","sea_water_speed (2.5 - 22 m), sea_water_to_dir...",Damaged by Hurricane Irma. A new oceanographic...,Julio Morell \n(julio.morell@upr.edu),False,POINT (-65.15680 18.47610)
8,"CARICOOS, OAP",MAPCO2,NCEI Accession 0117354,La Parguera MapCO2 buoy,NCEI Accession 0117354,17.953666666666667,-66.0512,Moored buoy,2009 - present,Operational,...,Yp,PMEL,Academia,CARICOOS,PMEL/CARICOOS,"SST, SSS, Atm. press, xCO2 water, xCO2 air, fC...",NaN,Julio Morell \n(julio.morell@upr.edu),False,POINT (-66.05120 17.95367)
9,CARICOOS,PUNTAS,PTRP4,"Rincón, PR",NaN,18.3667,-67.2514,Fixed,11/2012 - present,Offline,...,Yf,CARICOOS,Academia/Non-Profit,CARICOOS,NDBC / CARICOOS,"wind_speed (15 m), wind_speed_of_gust (15 m), ...",Issues with the data logger. CARICOOS will rep...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-67.25140 18.36670)
16,CARICOOS,XAMA,TIMA,"Charlotte Amalie Harbor, USVI",NaN,18.3303,-64.9264,Fixed,2009 - 2015,No,...,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",NaN,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-64.92640 18.33030)
27,CARICOOS,XSAV,TIAC,"Savanah Island, USVI",NaN,18.33737,-65.083,Fixed,2009 - 2017,No,...,N,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Decomissioned by Weatherflow because it got da...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-65.08300 18.33737)
28,CARICOOS,XWGO,TIGO,"Crown Mountain, USVI",NaN,18.35537,-64.96693,Fixed,2009 - 2017,No,...,Yf,CARICOOS,Industry,CARICOOS,Weatherflow /CARICOOS,"wind_speed (10 m), wind_speed_of_gust (10 m), ...",Damaged by Hurricane Maria. A new station will...,Patricia Chardón-Maldonado (patricia.chardon@u...,False,POINT (-64.96693 18.35537)


Print out the available metadata from the IOOS Catalog for the last dataset.

In [22]:
dataset

{'author': None,
 'author_email': None,
 'creator_user_id': 'bb55ba1f-9c92-4539-a676-e1fea85df891',
 'id': '47b02699-e375-434a-89b3-4c76366879c0',
 'isopen': False,
 'license_id': None,
 'license_title': None,
 'maintainer': None,
 'maintainer_email': None,
 'metadata_created': '2023-01-18T23:29:49.495839',
 'metadata_modified': '2024-04-08T17:06:07.138207',
 'name': 'xwgo-crown-mountain-usvi-aggregated-meteorological-observations',
 'notes': 'Realtime-Historical aggregated dataset from the Mesonet meteorological station network funded by CARICOOS and operated by Weatherflow Inc.',
 'num_resources': 4,
 'num_tags': 11,
 'organization': {'id': '2c66095c-e48c-4dff-9f82-c6427926c81a',
  'name': 'caricoos',
  'title': 'CariCOOS',
  'type': 'organization',
  'description': 'CARICOOS is the Regional Association of the national Integrated Ocean Observing System (IOOS®) in the US Caribbean, primarily Puerto Rico and the US Virgin Islands. We provide data and products to inform decision-making 

In [23]:
# GLOS special case as they provide ERDDAP ID

# for dataset in gdf_final.loc[(~gdf_final['erddap_avail'] & ~gdf_final['Dataset (ERDDAP) ID'].isna()),'Dataset (ERDDAP) ID'].tolist():
#     print(dataset)
#     e.dataset_id = dataset
    
#     e.constraints = {
#                  "time>=": "{}-01-01".format(cy)
#             }
        
#     e.variables = ['time']
    
#     try:
        
#         df_data = e.to_pandas(parse_dates=True)
        
#         string = '\tDate range: {} - {}'.format(df_data.min()[0],df_data.max()[0])
        
#         print(string)
        
#         if pd.to_datetime(df_data.max()[0], utc=True) > pd.to_datetime('{}-01-01'.format(cy), utc=True):
            
#             datasetid = gdf_final.loc[gdf_final['Dataset (ERDDAP) ID']==dataset,'Station ID'].tolist()[0]
            
#             problem_datasets.remove(datasetid)
        
#     except:
#         print('\tNo valid data')

In [24]:
#gdf_ckan.loc[gdf_ckan['Station ID'].isin(problem_datasets)]